In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display
from sklearn.metrics import accuracy_score,confusion_matrix, ConfusionMatrixDisplay, classification_report
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
def XGBoost(df):
    X = df.drop('diagnostico', axis=1)
    y = df['diagnostico']
    # Substituir valores infinitos por NaN
    #X.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Tratar valores NaN, preenchendo com a média da coluna
    #X.fillna(X.mean(), inplace=True)

    # Verificar e tratar valores extremamente grandes (se necessário)
    #max_value = np.finfo(np.float64).max
    #X = X.clip(lower=-max_value, upper=max_value)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

    # Aplicar SMOTE ao conjunto de treinamento
    smote = SMOTE(random_state=1)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    params_grid = {
        'learning_rate': [0.01],
        'gamma': [0, 0.1],
        'n_estimators': [100, 200],
        'max_depth': [3, 4, 5],
        'min_child_weight': [2, 3, 4],
        'subsample': [0.6, 0.8, 1],
        'colsample_bytree': [0.5, 0.7],
        'scale_pos_weight': [1, 2]
    }

    classifier = xgb.XGBClassifier()

    grid_classifier = GridSearchCV(classifier, params_grid, scoring='accuracy', cv=5)
    grid_classifier.fit(X_val, y_val)
    best_parameters = grid_classifier.best_params_
    
    model = xgb.XGBClassifier(**best_parameters)
    model.fit(X_train_smote, y_train_smote)
    pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, pred)
    print(classification_report(y_test,pred))
    print(confusion_matrix(y_test, pred))

    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, pred), annot=True, fmt='d', cmap='Blues', xticklabels=['Sem Sepse', 'Com Sepse'], yticklabels=['Sem Sepse', 'Com Sepse'])
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.title('Matriz de Confusão')
    plt.show()

Valores normais

In [ ]:
df_normal = pd.read_csv('df_normal.csv')
XGBoost(df_normal)

KNN

In [ ]:
df_knn = pd.read_csv('df_knn.csv')
XGBoost(df_knn)

Media

In [ ]:
df_media = pd.read_csv('df_media.csv')
XGBoost(df_media)

Mediana

In [ ]:
df_mediana = pd.read_csv('df_mediana.csv')
XGBoost(df_mediana)

Moda

In [ ]:
df_moda = pd.read_csv('df_moda.csv')
XGBoost(df_moda)

Regressão

In [ ]:
df_regressao = pd.read_csv('df_regressao.csv')
XGBoost(df_regressao)